In [ ]:


# Directories
drunk_dir = '/content/drive/MyDrive/NeurIPS/Code/cropped/drunk'
sober_dir = '/content/drive/MyDrive/NeurIPS/Code/cropped/sober'

# Load images and labels
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(label)
    return images, labels

drunk_images, drunk_labels = load_images_from_folder(drunk_dir, 1)  # Label 1 for drunk
sober_images, sober_labels = load_images_from_folder(sober_dir, 0)  # Label 0 for sober




58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
print(len(drunk_labels))
print(len(sober_labels))

allImages = []
allLabels = []

for i in range(len(drunk_images)):
  allImages.append(drunk_images[i])
  allImages.append(sober_images[i])
  allLabels.append(drunk_labels[i])
  allLabels.append(sober_labels[i])





# Combine and split the dataset
images = np.array(allImages)
labels = np.array(allLabels)

# Normalize images
images = images.astype('float32')
images /= 255.0


4670
4670


In [ ]:
# Split the dataset
train_x, temp_x, train_y, temp_y = train_test_split(images, labels, test_size=0.3, random_state=42, shuffle=False)
val_x, test_x, val_y, test_y = train_test_split(temp_x, temp_y, test_size=0.5, random_state=42, shuffle=False)

# Data generators
train_datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(train_x, train_y, batch_size=32)
val_generator = val_datagen.flow(val_x, val_y, batch_size=32)
#test_generator = test_datagen.flow(test_x, test_y, batch_size=32)

# Load VGG16 model
base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the base_model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = layers.Flatten()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(1, activation='sigmoid')(x)  # Binary classification (drunk or sober)

model = Model(inputs=base_model.input, outputs=x)


In [ ]:
import numpy as np
import tensorflow as tf
import os
import cv2
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision, F1Score
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy', Recall(), Precision()])
model.summary()

# Train the model
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(train_generator, epochs=30, validation_data=val_generator, callbacks=[early_stopping])

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
# Evaluate the model
test_loss, test_acc, test_recall, test_precision = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')
print(f'Test Recall: {test_recall}')
print(f'Test Precision: {test_precision}')

48/48 [==============================] - 6s 130ms/step - loss: 0.4099 - accuracy: 0.8161 - recall_6: 0.8610 - precision_6: 0.8092
Test accuracy: 0.8161375522613525
Test Recall: 0.8610085844993591
Test Precision: 0.8092485666275024


In [ ]:
print(len(test_x))

1401
